# AIPROD Phase 1 Training - Google Colab
## Configuration et lancement automatique

### Étape 1: Monter Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
print("✓ Google Drive monté avec succès!")

### Étape 2: Télécharger et extraire le repository

In [ ]:
import os
import subprocess
import shutil

%cd /content

# Nettoyer les anciens fichiers
for item in ['AIPROD', 'AIPROD-main', 'main.zip']:
    path = f'/content/{item}'
    if os.path.exists(path):
        if os.path.isdir(path):
            shutil.rmtree(path)
        else:
            os.remove(path)
        print(f"  ✓ Supprimé: {item}")

print("✓ Nettoyage effectué\n")

# Télécharger le ZIP
print("⏳ Téléchargement du repository...")
result = subprocess.run(['wget', '-q', 'https://github.com/Blockprod/AIPROD/archive/refs/heads/main.zip'], 
                       capture_output=True, text=True)
if result.returncode == 0:
    print("✓ Téléchargement réussi")
else:
    print(f"⚠️ Erreur: {result.stderr}")

# Vérifier que le ZIP existe
if os.path.exists('/content/main.zip'):
    print(f"✓ Fichier ZIP détecté: {os.path.getsize('/content/main.zip') / 1e6:.1f} MB")
    
    # Extraire
    print("⏳ Extraction du repository...")
    result = subprocess.run(['unzip', '-q', 'main.zip'], capture_output=True, text=True)
    if result.returncode == 0:
        print("✓ Extraction réussie")
    else:
        print(f"⚠️ Erreur extraction: {result.stderr}")
    
    # Vérifier et renommer
    if os.path.exists('/content/AIPROD-main'):
        print("✓ Répertoire AIPROD-main détecté")
        shutil.move('/content/AIPROD-main', '/content/AIPROD')
        print("✓ Répertoire renommé en AIPROD")
    else:
        print("⚠️ AIPROD-main non trouvé après extraction")
        print(f"Répertoires présents: {os.listdir('/content/')}")
else:
    print("⚠️ Le ZIP n'a pas pu être téléchargé!")

# Vérification finale
%cd /content/AIPROD

if os.path.exists('/content/AIPROD/requirements.txt'):
    print("\n✅ Repository prêt!")
    print(f"   Chemin: {os.getcwd()}")
    print(f"   Fichiers: {len(os.listdir())} items")
else:
    print("\n⚠️ ERREUR: requirements.txt non trouvé")
    print(f"   Chemin actuel: {os.getcwd()}")
    print(f"   Contenu: {os.listdir('.')[:10]}")

### Étape 3: Vérifier requirements.txt

In [ ]:
import os

%cd /content/AIPROD

# Vérifier la présence de requirements.txt
req_file = '/content/AIPROD/requirements.txt'

print("⏳ Vérification du repository...")
print(f"   Chemin: {os.getcwd()}")
print(f"   requirements.txt existe: {os.path.exists(req_file)}")

if os.path.exists(req_file):
    print("\n✓ requirements.txt trouvé")
    with open(req_file, 'r') as f:
        lines = f.readlines()
    print(f"✓ Contient {len(lines)} dépendances")
    print("\nPremières dépendances:")
    for line in lines[:5]:
        print(f"  - {line.strip()}")
else:
    print("\n⚠️ ERREUR: requirements.txt non trouvé!")
    print(f"\nRépertoire courant: {os.getcwd()}")
    print(f"Fichiers présents:")
    for f in sorted(os.listdir())[:15]:
        print(f"  - {f}")
    
    # Chercher le fichier ailleurs
    for root, dirs, files in os.walk('/content'):
        if 'requirements.txt' in files:
            print(f"\n✓ Trouvé: {os.path.join(root, 'requirements.txt')}")
            break

### Étape 4: Installer PyTorch et dépendances

In [ ]:
import subprocess
import sys

%cd /content/AIPROD

print("="*60)
print("INSTALLATION DES DÉPENDANCES")
print("="*60)

# Fonction pour installer avec gestion d'erreur
def install_package(package_spec, label):
    print(f"\n⏳ {label}...")
    result = subprocess.run(
        [sys.executable, '-m', 'pip', 'install', '-q'] + package_spec.split(),
        capture_output=True,
        text=True
    )
    if result.returncode == 0:
        print(f"✓ {label} réussi")
        return True
    else:
        print(f"⚠️ Erreur: {result.stderr[:200]}")
        return False

# PyTorch
install_package('torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118', 
                'Installation de PyTorch')

# Requirements du projet
try:
    result = subprocess.run(
        [sys.executable, '-m', 'pip', 'install', '-q', '-r', 'requirements.txt'],
        capture_output=True,
        text=True,
        timeout=300
    )
    if result.returncode == 0:
        print("✓ requirements.txt installé")
    else:
        print(f"⚠️ Erro requirements.txt: {result.stderr[:300]}")
except subprocess.TimeoutExpired:
    print("⚠️ Installation des requirements interrompue (timeout)")
except Exception as e:
    print(f"⚠️ Erreur: {e}")

# Packages supplémentaires
install_package('peft transformers accelerate tensorboard', 
                'Installation des packages supplémentaires')

print("\n" + "="*60)
print("✅ Installation terminée!")
print("="*60)

### Étape 5: Préparer les checkpoints dans Google Drive

In [ ]:
import os

# Créer un dossier pour les checkpoints dans Google Drive
checkpoint_dir = '/content/drive/MyDrive/AIPROD_checkpoints'
os.makedirs(checkpoint_dir, exist_ok=True)

print(f"✓ Dossier créé: {checkpoint_dir}")
print(f"✓ Les checkpoints seront sauvegardés automatiquement dans Google Drive")
print(f"\nPour restaurer les checkpoints après une déconnexion:")
print(f"  !cp /content/drive/MyDrive/AIPROD_checkpoints/* /content/AIPROD/checkpoints/")

### Étape 6: Vérifier le GPU disponible

In [ ]:
import torch

print("=== VÉRIFICATION GPU ===")
print(f"✓ PyTorch version: {torch.__version__}")
print(f"✓ CUDA disponible: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"✓ GPU détecté: {torch.cuda.get_device_name(0)}")
    print(f"✓ VRAM disponible: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
    print(f"✓ Compute Capability: {torch.cuda.get_device_capability(0)}")
else:
    print("⚠️ ATTENTION: Aucun GPU détecté!")
    print("   Assurez-vous que Runtime → Change runtime type → GPU est sélectionné")

### Étape 7: Démarrer l'entraînement Phase 1

In [ ]:
%cd /content/AIPROD

print("="*50)
print("🚀 DÉMARRAGE DE L'ENTRAÎNEMENT PHASE 1")
print("="*50)
print()

!python packages/aiprod-core/src/training/train.py --start-phase 1 --max-epochs 10

### Étape 8: Sauvegarder les checkpoints dans Google Drive

In [ ]:
import os
import shutil

src = '/content/AIPROD/checkpoints'
dst = '/content/drive/MyDrive/AIPROD_checkpoints'

if os.path.exists(src):
    files = os.listdir(src)
    print(f"✓ {len(files)} fichiers de checkpoint détectés")
    
    for file in files:
        src_file = os.path.join(src, file)
        dst_file = os.path.join(dst, file)
        shutil.copy2(src_file, dst_file)
        print(f"  ✓ Sauvegardé: {file}")
    
    print(f"\n✅ Tous les checkpoints sauvegardés dans Google Drive!")
else:
    print("⚠️ Dossier checkpoints non trouvé")

### Étape 9: Monitorer le GPU en temps réel (optionnel)

In [ ]:
import subprocess
import time
from datetime import datetime

print("Monitoring GPU (Ctrl+C pour arrêter)")
print("="*70)
print("Heure      | Temp(C) | Utilisation(%) | Puissance(W) | GPU Clock(MHz)")
print("-"*70)

try:
    while True:
        try:
            result = subprocess.run(
                ['nvidia-smi', '--query-gpu=temperature.gpu,utilization.gpu,power.draw,clocks.current.graphics', 
                 '--format=csv,noheader,nounits'],
                capture_output=True,
                text=True,
                timeout=5
            )
            
            if result.returncode == 0:
                values = result.stdout.strip().split(', ')
                if len(values) >= 4:
                    temp = values[0].strip()
                    util = values[1].strip()
                    power = values[2].strip()
                    clock = values[3].strip()
                    
                    timestamp = datetime.now().strftime('%H:%M:%S')
                    print(f"{timestamp}   | {temp:>5} | {util:>13} | {power:>11} | {clock:>13}")
        except Exception as e:
            print(f"Erreur lecture GPU: {e}")
        
        time.sleep(10)
        
except KeyboardInterrupt:
    print("\n✓ Monitoring arrêté")

## ✅ C'est tout!

Le notebook exécutera automatiquement:
1. ✓ Montage de Google Drive
2. ✓ Téléchargement du repository
3. ✓ Installation de toutes les dépendances
4. ✓ Vérification du GPU
5. ✓ Lancement de l'entraînement Phase 1
6. ✓ Sauvegarde des checkpoints dans Google Drive

**Comment utiliser:**
1. Copie le contenu de ce notebook dans Google Colab
2. Ou ouvre directement le fichier `.ipynb` depuis Google Drive
3. Assure-toi que Runtime → Change runtime type → GPU est sélectionné
4. Exécute les cellules dans l'ordre (Ctrl+Enter)